### Applied Machine Learning
# HW4

### Shuai Hao (sh3831), Eugene M. Joseph (emj2152)

## Predict wine quality from review texts
This notebook explores models using feature vectors from a trained BERT model using [the wine reviews data](https://www.kaggle.com/zynicide/wine-reviews) from Kaggle. The data were scraped on November 22nd, 2017. For this task, only the wine from the US are used.

## Task 3 Transformers (bonus / optional) [50pts]
Fine-tune a BERT model on the text data alone using the transformers library.
How does this model compare to a BoW model, and how does it compare to a model using all features?

Although the prompt for this questions asks us to fine-tune a BERT model, we saw the post on Piazza where Professor Mueller said he was just expecting us to use the pretrained model from HuggingFace, so that's what we've done in this notebook.    

Piazza Post Link: https://piazza.com/class/k5phinua89310n?cid=439  
**Here's the relevant part**: "The goal was to use the features as-is without finetuning for this exercise. -AM"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from operator import itemgetter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, Normalizer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score, f1_score
import seaborn as sns
import spacy
from transformers import pipeline
import time

I0421 22:57:26.878679 4603985344 file_utils.py:57] TensorFlow version 2.0.0-rc0 available.


In [42]:
df_all = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

In [43]:
df = df_all[df_all["country"] == "US"]

In [44]:
df['all_text'] = df['title'].astype(str) + " " +df['description'].astype(str)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


The reduce the running time for this part, we sampled 35% percent of the data

In [45]:
df = df.sample(frac=0.35, replace=True, random_state=1)

In [89]:
y = df['points']
X = df['all_text']

In [47]:
nlp = pipeline("feature-extraction")

I0420 17:21:12.993907 4385304000 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-cased-config.json from cache at /Users/emjoseph/.cache/torch/transformers/774d52b0be7c2f621ac9e64708a8b80f22059f6d0e264e1bdc4f4d71c386c4ea.62ec375c5c2c4c3afe8f83d619fc9000f594972092c9113a1747a472a9936c17
I0420 17:21:12.999413 4385304000 configuration_utils.py:319] Model config DistilBertConfig {
  "_num_labels": 2,
  "activation": "gelu",
  "architectures": null,
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "length_pena

I0420 17:21:13.688190 4385304000 file_utils.py:489] storing https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-cased-modelcard.json in cache at /Users/emjoseph/.cache/torch/transformers/e7d48e6fc39207404b4bb02c7e5b18c589596b890807864fe4b46fcd88e54e28.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331
I0420 17:21:13.691084 4385304000 file_utils.py:492] creating metadata file for /Users/emjoseph/.cache/torch/transformers/e7d48e6fc39207404b4bb02c7e5b18c589596b890807864fe4b46fcd88e54e28.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331
I0420 17:21:13.693060 4385304000 filelock.py:318] Lock 5754136464 released on /Users/emjoseph/.cache/torch/transformers/e7d48e6fc39207404b4bb02c7e5b18c589596b890807864fe4b46fcd88e54e28.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock
I0420 17:21:13.693679 4385304000 modelcard.py:161] loading model card file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-cased-modelcard.js

I0420 17:21:13.985244 4385304000 modeling_tf_utils.py:390] loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-cased-tf_model.h5 from cache at /Users/emjoseph/.cache/torch/transformers/b682f767644ff4a33fb4b68b33e7616d9755137bc6698bd629a0d2ab1ed3883f.e9a7fccc6d3a446e28bd3e5b94a83c3fefa8f745a33ab861db2f2b05f201ab37.h5
I0420 17:21:15.729577 4385304000 modeling_tf_utils.py:432] Layers from pretrained model not used in TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']


In [90]:
X

79452     Koehler 2009 Dry Riesling (Santa Ynez Valley) ...
29434     The Magpie 2009 Lot Five Merlot (Bennett Valle...
12550     Morgan 2013 Metallico Un-Oaked Chardonnay (Mon...
78288     Renwood 2012 Red (Amador County) This wine sho...
119619    Vine Cliff 2004 Cabernet Sauvignon (Napa Valle...
104693    Veramar 2010 Viognier (Virginia) Aromas of toa...
18857     Heron 2013 Pinot Noir (Monterey County) Globe-...
124337    Keegan 2000 Chardonnay (Russian River Valley) ...
51686     Calera 2012 Viognier (Mt. Harlan) As luscious ...
79210     Migration 2013 Searby Vineyard Chardonnay (Rus...
49749     Early Mountain 2013 Reserve Chardonnay (Virgin...
117374    Animale 2014 Maria Carmela Red (Washington) Th...
18703     Buena Vista 2013 Arpad's Selection Zinfandel (...
104773    Thunder Mountain 1999 Ciardella Vineyard Chard...
75262     Alpha Omega 2012 Sauvignon Blanc (Napa Valley)...
87176     Migration 2015 Bien Nacido Vineyard Chardonnay...
77241     Bruliam 2014 Soberanes Vineyar

In [91]:
X_bert = [np.squeeze(np.average(np.array(nlp(t)),axis=1), axis=0) for t in X]

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X_bert, y, random_state=1)

In [93]:
scores = cross_val_score(LinearRegression(), X_train, y_train)
np.mean(scores)

0.6555470917138552

In [96]:
model_linear_regression = make_pipeline(LinearRegression())

In [97]:
model_linear_regression.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [98]:
model_linear_regression.score(X_test, y_test)

0.6709984511745875

The BERT model produced the best results compared to the Word2Vec, Glove, and FastText models that only used the text features. But our BOW+Word2Vec+Non-text features model and our BOW+Word2Vec models were still better.

The next step would be to combine the BERT + BOW + non-text features which could possibly produce the best model

Another important point is that the BERT model was not fine-tuned to our text data which contained lots of jargon and vocabulary centered around wine. Fine-tuning the BERT model may produce best performing and lightest model of all.

**Attempts at fine-tuning the BERT model below. Not completed.**

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

In [0]:
df_all = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

In [0]:
df = df_all[df_all["country"] == "US"]

In [0]:
df['all_text'] = df['title'].astype(str) + " " +df['description'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
y = df['points'].values
X = df['all_text']

In [0]:
MAX_LEN = 128

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
X_ids = [tokenizer.encode(doc, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True) for doc in X]

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X_ids, y, random_state=1)

In [0]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_valid = torch.tensor(X_valid)
y_valid = torch.tensor(y_valid)

In [0]:
batch_size = 8

In [0]:
train_data = TensorDataset(X_train, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(X_valid, y_valid)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels= 1).to(device)
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels= 1)

# Parameters:
lr = 2e-5
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr, eps=adam_epsilon, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
criterion = torch.nn.MSELoss()  # loss function

In [0]:
## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for e in range(epochs):
  print(f'Running {e} epochs...')
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_labels = batch

    # print(b_input_ids)
    # print(b_labels)
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, labels=b_labels.float())
    # print(outputs[0])
    #loss = criterion(outputs, b_labels)
    loss = outputs[0]
    
    # Backward pass
    loss.backward()
    
    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()
    
    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_r2_score, nb_eval_steps = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids,b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None)
    
    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = logits.flatten()
    labels_flat = label_ids.flatten()

    tmp_eval_r2_score = r2_score(labels_flat,pred_flat)
    
    eval_r2_score += tmp_eval_r2_score
    nb_eval_steps += 1

  print(F'\n\tR2 Score: {eval_r2_score/nb_eval_steps}')

Running 0 epochs...

	Current Learning rate:  1.3333333333333333e-05

	Average Training loss: 1155.7368974479211

	R2 Score: -0.25443925491414565
Running 1 epochs...
